# Getting Started with COP Model Network Analysis

## Install COPModel-NA

Highlight the following command and press Ctrl+C to copy it.

`git clone https://github.com/LetsCodePhysics/COPModel-NA.git`

Click on `New` $\rightarrow$ `Terminal`. Paste the command into the terminal using Ctrl+Shift+V.

Press Enter.

This should create a new folder called COPModel-NA. You can close the terminal tab.

## Import Libraries

Run the code cell below to get started by clicking inside the code cell and pressing Ctlr+Enter. You shouldn't need to change anything about it. Scroll past it and continue reading at "COP Model Networks".

In [ ]:
# Import libraries.

import sys
sys.path.insert(0, 'COPModel-NA/') 
from copmodelna import *

import warnings
warnings.filterwarnings('ignore')
import datetime
import networkx as nx
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import math
import numpy as np
import matplotlib.pyplot as plt

## COP Model Networks

This Jupyter notebook reads in a spreadsheet of data from student COP model drawings and produces a set of **network diagrams**. In a network diagram...

* Each circle (**node**) represents one drawing element.
* A node's **size** represents the element's **frequency**.
* Nodes are **color-coded** based on COP **categories**.
* Lines (**edges**) represent two elements occuring in the **same drawing**.
* A line's **thickness** represents **frequency of co-incidence**.

The spreadsheet must be a Google Sheet formatted with columns as follows:
```
Element,Category,Drawing 1,Drawing 2,...
```
`Element` = List of individual elements found in the COP models. These are text cells.

`Category` = The type of COP element. This must be Member, Practice, Goal, or Demographic (used to filter student populations). Any Demographic items must be included in the first M rows of the database; they cannot appear below Member, Practice, or Goal elements.

`Drawing N` = Did drawing N include this element? If yes, the cell = y. If no, it's empty. (For demographic lines, this cell includes demographic info.)

Here is the link to our spreadsheet for instructional labs: https://docs.google.com/spreadsheets/d/1TGk3QrpYbnSmVDT-c52PDL_ysp115mEm_mloAbwPRIs/edit?gid=0#gid=0

## Read in the spreadsheet.

To get started, you need to **read in the spreadsheet**. Run the code cell below.

*If you need to use a different spreadsheet* for a different set of data, just replace the Google Sheet ID located in `https://docs.google.com/spreadsheets/[Google Sheet ID goes here]/export` with your own. Note that the data will be pulled from the *leftmost tab* in the spreadsheet.

In [ ]:
# Read in the spreadsheet.

full_database = pd.read_excel("https://docs.google.com/spreadsheets/d/1TGk3QrpYbnSmVDT-c52PDL_ysp115mEm_mloAbwPRIs/export",engine='openpyxl')#,sheet_name=sheet_name)

## Create a network to study.

The next step is to **create a list** of the drawings you want to include in your analysis. To do this, use the `MakeSubgroup` function. This function requires at least one input argument: `full_database` is the name of the database created by calling `pd.read_excel` above.

By default, `MakeSubgroup(full_database)` will create a network using **all** the drawings in `full_database`. You can add more input arguments to `MakeSubgroup` to **filter** the drawings in the network. Just follow the pattern `MakeSubgroup(full_database,'row name 1','value 1','row name 2','value 2')` where...
* `'row name'` is the name of a **row** in the spreadsheet (such as `'Gender'`, `'Lab Name'`, or the name of an element).
* `'value'` is the **value** you want the drawings in your network to have (such as `'female'`, `'ipls'`, or `'y'` to check for an element).

Here are a few examples of using `MakeSubgroup`:
* `ipls_drawings = MakeSubgroup(full_database,'Lab Name','ipls')` will create a list of all drawings about the ipls lab, and store that list under the variable name `ipls_drawings`.
* `ipls_female_drawings = MakeSubgroup(full_database,'Lab Name','ipls','Gender','female')` will create a list of all drawings about the ipls lab made by female students, and store that list under the variable name `ipls_female_drawings`.
* `ipls_firstgen_drawings = MakeSubgroup(full_database,'Lab Name','ipls','First Gen',True)` will create a list of all drawings about the ipls lab made by first-generation college students, and store that list under the variable name `ipls_firstgen_drawings`. (Note that, in our spreadsheet, `'First Gen'` values are boolen True or False, not strings, and therefore the `True` in this example does not have quotation marks around it.

Once you have these lists, you can **manipulate** them using [Python list operations](https://www.geeksforgeeks.org/python-lists/). For example, suppose you have a reason for **excluding** Drawing 16 from your analysis of `ipls_female_drawings`. You can remove it using `ipls_female_drawings.remove('Drawing 16')`. 

Or suppose you want to **combine** two lists, like `ipls_female_drawings` and `ipls_firstgen_drawings`. Since these lists likely have some drawings in common (i.e., drawings from first-generation female students), we can use `combined_list = list(set(ipls_female_drawings).union(ipls_firstgen_drawings))` to combine the lists and avoid duplicates. Here, the `set` and `union` functions take in a list and produce a set of items with no duplicates. The `list` function then turns the resulting set back into a list.

Or suppose you want to create a list of drawings that **does include the drawings** in another list. For example, suppose you had a list `male_ipls_drawings = MakeSubgroup(full_database,'Lab Name','ipls','Gender','male')` and wanted all the drawings from non-male students. You can use `nonmale_ipls_drawings = list(set(ipls_drawings) - set(male_ipls_drawings))`. Here, the `-` operator removes items the two sets have in common.

In [ ]:
# Create a list for your network.

my_list = MakeSubgroup(full_database)

## Create and visualize the graph.

The list you just created tells Python **which drawings to include in a graph**. A graph is a Python object that stores information about nodes and edges. To create a graph object from your list of drawings, use `G_name = MakeGraph(my_list,full_database)`. Here, `my_list` is the list of drawings you created, and `full_database` is the name of the database created by calling `pd.read_excel` above. Make sure to give your graph a name that you can associate with the list (something more specific than `G_name` here).

Creating the graph can take some time, so you should usually run this function in its own code cell.

After you create the graph, create a **network diagram** of it using the `DrawGraph(G_name)` function. `DrawGraph` requires one input, the name of the graph you created using `MakeGraph`. It will produce a graphic of the network diagram in the output below the code cell.

If your network diagram is difficult to examine (for example, the nodes are too small or too big), you can also provide two optional arguments: `DrawGraph(G,node_size_control=0.75,edge_size_control=1.0)`. Here, `node_size_control` adjusts the size of the nodes and `edge_size_control` adjusts the size of the edges. Because these arguments are optional, you need to include the variable names and an equal sign before each number like in this example.

Also note that Jupyter **can't display more than one network diagram from each code cell**. This means if you use `DrawGraph` multiple times in a single code cell, it will show only the last one.

Once you have your network diagram, you can right-click and save the image to your computer for use in a paper, email, etc.

In [ ]:
# Create a graph.

G_name = MakeGraph(my_list,full_database)

In [ ]:
# Create network diagram.

DrawGraph(G_name)

## Analyze the graph.

*last updated 6Feb2025*

Once you have your graph made (let's keep calling it `G_name`), there are many functions you can use to analyze it. Which functions you use depends on your research question, but here are the ones I've found most useful so far. Keep an eye out for updates in this section as we add more functions.

* `clusters = DetectClusters(G_name,weight='weight',method='method')` This function partitions your graph into **clusters**. ("Clusters" are sometimes called "communities" in the literature, but we're already using the word "community" to refer to a community of practice. Just keep that in mind while you compare our work with the literature.) A cluster is a set of nodes that are more connected to each other than they are to other nodes in the graph. There are many methods you can specify in this function; we use `'fast-greedy'` as a default for now but that might change. The output `clusters` is a set of lists, with each list containing the nodes in a given cluster.

* `nx.density(G)` will give the **density** of graph `G`. This value ranges from 0 to 1 and represents how well connected the nodes are compared to how connected they could be.

* `G.degree('node')` will give the degree of the node named `'node'` in graph `G`. This gives you a number that tells you how many edges come out of the node named `'node'`.

* `NodeStrength(G,'node')` is a number equal to the sum of all the edge weights coming out of the node named `'node'` in graph `G`.

* `nx.betweenness_centrality(G)['node']` will give the fraction of times node `'n'` acts as a bridge along the shortest path linking two other nodes in the graph `'G'`. This returns a number between 0 and 1. If you add the argument `weight = 'weight_inverse'` after the `'G'`, it will weight these paths by 1 / the edge weight, making stronger connections a shorter distance.

* `NodeDegreeCosine(G1,G2)` will compare the node degrees between graphs `'G1'` and `'G2'`. It returns a value between 0 and 1. If the two graphs are similar, the value is closer to 1. If the two graphs are different, the value is closer to 0.

* `NodeStrengthCosine(G1,G2)` will compare the node strengths between graphs `'G1'` and `'G2'`. It returns a value between 0 and 1. If the two graphs are similar, the value is closer to 1. If the two graphs are different, the value is closer to 0.

* `NodeWeightCosine(G1,G2)` will compare the node weights between graphs `'G1'` and `'G2'`. It returns a value between 0 and 1. If the two graphs are similar, the value is closer to 1. If the two graphs are different, the value is closer to 0.

* `EEJ(G1,G2)` will compare the existence of edges between graphs `'G1'` and `'G2'`. It returns a value between 0 and 1. If the two graphs are similar, the value is closer to 1. If the two graphs are different, the value is closer to 0.

* `AllStrength(G)` is the total of all nodes' strengths (from `'NodeStrength'`) across network `'G'`. This is also the sum of all y's in the original database.

* `Modularity(G,clusters)` is a number between -1 and 1 that measure of how well the graph `'G'` was separated into the list of clusters you obtained from `DetectClusters`.

* `PurityOfClustering(clusters,other_clusters)` takes in two lists of clusters you obtained from `DetectClusters`, each for a different graph, and gives a number between 0 and 1 telling you how much overlap there is between the two clusters (0 being none, and 1 being a perfect match). We will usually run `PurityOfClustering` with arguments `(G_all_clusters,G_subset_clusters)` where `G_all_clusters` is the list of clusters for the full data set and `G_subset_clusters` is the list of clusters for the subset of data you're exploring.

* `FMeasure(clusters,other_clusters)` is similar to purity ([citation](https://www.sciencedirect.com/science/article/pii/S0378873321000307?casa_token=Y4KM5fBX43MAAAAA:ivqcUlFsXD2HZwQ5_QMvKApMMx85F0kiFqWw5PQCYANsXKqLh0_AzIx1BHbm3KS0m_Z5UpDNvc4#:~:text=2.3.2.-,F%2DMeasure,-The%20F%2DMeasure)).

* `BootStrapTest(drawings_in,full_database,N,threshold=0.50,print_output=True,file_out='bootstrap_output.txt')` will **test the stability of your clusters** using iterative bootstraps. A **bootstrap** is a copy of your network where the edge weights have been randomized using a Poisson distribution. **This function takes a while to run** so you should run it and plan to come back later. The input arguments it takes are...
  * `drawings_in` = The list of drawings you want to include.
  * `full_database` = The database you read in using `pd.read_excel`.
  * `N` = The number of bootstrap iterations to use. Our dataset usually requires N = 1200.
  * `threshold` (optional) = The % of the time you want to check for nodes to be assigned to the same cluster.
  * `print_output` (optional) = True if you want to see an update printed to the screen at the end of each iteration; False if you don't.
  * `file_out` (optional) = The name of a text file that this function will write its output to. This is necessary since `BootStrapTest` takes so long to run you might need to close the Jupyter notebook before it finishes.
  
  `BootStrapTest` will also write out much of its output to a **dictionary** if you want to use it in this notebook. So, you can use `output = BootStrapTest(...)` to save that dictionary under the variable name `output`.

In [ ]:
# Use analysis functions here.


## Compare two (really, three) networks.

`BootStrapComparison(all_drawings,drawing_subset_1,drawing_subset_2,full_database,N,N_nodes=5,file_out='bootstrapcomparison.txt',time_print=False,centrality_power=2)` is the largest function in the library. It runs an **iterative bootstrap comparison between the graphs of two subgroups**, using as a baseline the full dataset which these subgroups are drawn from. For example, `BootStrapComparison(ipls_drawings,female_ipls_drawings,male_ipls_drawings,full_database,N)` would run an `N`-iteration comparison between `female_ipls_drawings` and `male_ipls_drawings` using `ipls_drawings` as a baseline. Optional arguments include...
* `N_nodes` = number of largest nodes to compare between the two networks.
* `file_out` = name of text file to write output to
* `time_print` = True if you want to see how many seconds each step in the process takes. This is useful for diagnostic purposes.
* `centrality_power` = an integer used to calculate the generalized mean of centrality measures within a network: 

$$A_p = \left( \frac{1}{N}\sum_{i=1}^{N} x_{i}^{p} \right)^{1/p}$$.

**This function takes the longest to run** but it completes all steps of analysis we need to compare the drawings from two subpopulations.

In [ ]:
# Run BootStrapComparison here.

BootStrapComparison(all_drawings,drawing_subset_1,drawing_subset_2,full_database,N,N_nodes=5,file_out='bootstrapcomparison.txt',time_print=False,centrality_power=2

## When you need to update COPModel-NA...

Occaissionally we'll update the COPModel-NA library with new features. When you need to update the library, highlight the following command and press Ctrl+C to copy it.

`cd COPModel-NA ; git pull https://github.com/LetsCodePhysics/COPModel-NA.git`

Click on `New` $\rightarrow$ `Terminal`. Paste the command into the terminal using Ctrl+Shift+V.

Press Enter.

This should update the COPModel-NA library. You can close the terminal tab. Back in the Jupyter notebook, make sure you re-run the code cell where you have `from copmodelna import *` to update the library within the Jupyter notebook.